# Dubliners ASR

> "wav2vec2 words and phones"

- branch: master
- categories: [asr, wav2vec2, dubliners, kaggle]

Original [here](https://www.kaggle.com/code/jimregan/dubliners-asr) with (partial) output; see also [this notebook]({% post_url 2026-01-11-dubliners-asr-v2 %}).

In [ ]:
!pip -q install --no-deps pyctcdecode kenlm pygtrie

In [2]:
import os
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"

In [3]:
import json
from pathlib import Path

import torch
import librosa

from transformers import AutoProcessor, AutoModelForCTC, pipeline

EN_MODEL = "jonatasgrosman/wav2vec2-large-xlsr-53-english"
PHONE_MODEL = "jimregan/wav2vec2-xls-r-300m-phoneme-timit"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUT = Path("/kaggle/working")

pipe_en = pipeline("automatic-speech-recognition", model=EN_MODEL, device=0 if DEVICE=="cuda" else -1)
pipe_phone = pipeline("automatic-speech-recognition", model=PHONE_MODEL, device=0 if DEVICE=="cuda" else -1)

en_proc = AutoProcessor.from_pretrained(EN_MODEL)
en_model = AutoModelForCTC.from_pretrained(EN_MODEL).to(DEVICE).eval()

V1 = Path("/kaggle/input/download-dubliners/v1")
V2 = Path("/kaggle/input/download-dubliners/v2")

for base in (V1, V2):
    prefix = base.name  # "v1" / "v2"

    for file in base.rglob("*.mp3"):
        stem = f"{prefix}_{file.stem}"

        en_out = pipe_en(str(file), chunk_length_s=10, return_timestamps="word")
        phone_out = pipe_phone(str(file), chunk_length_s=10, return_timestamps="word")

        (OUT / f"{stem}_en.json").write_text(json.dumps(en_out, ensure_ascii=False), encoding="utf-8")
        (OUT / f"{stem}_phone.json").write_text(json.dumps(phone_out, ensure_ascii=False), encoding="utf-8")


2026-01-10 15:38:01.583560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768059481.823684      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768059481.894707      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768059482.447037      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768059482.447558      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768059482.447562      17 computation_placer.cc:177] computation placer alr